## GPU

In [ ]:
# TODO move to utils

In [ ]:
import CUDA
import Flux
import NNlibCUDA
import SparseArrays

In [ ]:
CUDA.math_mode!(CUDA.FAST_MATH; precision = :BFloat16)
NNlibCUDA.softmaxalgo() = NNlibCUDA.CUDNN_SOFTMAX_ACCURATE

In [ ]:
device(x) = Flux.gpu(x)
# efficiently convert a sparse cpu matrix into a dense CUDA array
device(x::SparseArrays.AbstractSparseArray) = CUDA.functional() ? CUDA.CuArray(gpu(x)) : collect(x)
device(x::Tuple) = device.(x)

In [ ]:
function device_free!(x)
    if !CUDA.functional()
        return
    end
    CUDA.unsafe_free!(x)
end
device_free!(x::Tuple) = device_free!.(x);